## PSTAT 134 Project

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#loading the dataset 
OgData = pd.read_csv('/home/jovyan/work/Grand-Finale/dataset/movie_metadata_cleaned.csv', header='infer')

In [ ]:
OgData.head()

In [ ]:
#dropping unnecessary columns 
OgData=OgData.drop(['id','imdb_id','status'],axis=1)

In [ ]:
# replacing 0 with np.nan 
# we notice that a large part of revenue has equal to 0
# replacing with NaN so the distribution isn't skewed.  
OgData[OgData['revenue']==0].shape
OgData['revenue']=OgData['revenue'].replace(0,np.nan)
OgData['budget']=OgData['budget'].replace(0,np.nan)

In [ ]:
#formatting csv columns that are in json/dict format and converting them to lists
# only run once or error
from ast import literal_eval
OgData['genres']=OgData['genres'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])
OgData['production_companies']=OgData['production_companies'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])
OgData['production_countries']=OgData['production_countries'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])
OgData['spoken_languages']=OgData['spoken_languages'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])
OgData['belongs_to_collection']=OgData['belongs_to_collection'].fillna('[]').apply(literal_eval).apply(lambda x:x['name'] if isinstance(x,dict) else np.nan)
# Adding in return where >1 is profit and <1 is loss
OgData['return']=OgData['revenue']/OgData['budget']


In [ ]:
#this url is not included in the poster_paths column
BaseUrl = 'http://image.tmdb.org/t/p/w185'

In [ ]:
# adding Base URl to every poster path
OgData['poster_path']=BaseUrl+OgData['poster_path']
OgData

In [ ]:
# image processing Test
from skimage import io
from skimage.color import rgb2gray
# had to convert image to gray or else the model won't run
x=io.imread(OgData.poster_path[0])
x=rgb2gray(x)
x

In [ ]:
io.imshow(x)

In [ ]:
#determining if any movies don't have th poster link
OgData['poster_path'].isnull().sum()

In [ ]:
OgData[OgData['poster_path'].isnull()].index

In [ ]:
# reading in actual pictures
Poster=[]
for i in range(len(OgData)):
    #accounting for the empty links
    if i==2407 or i==2429:
        next
    else:
        y=io.imread(OgData.poster_path[i])
        Poster.append(rgb2gray(y))
    

In [ ]:
from skimage.io._plugins import matplotlib_plugin

In [ ]:
#preview first 6
import matplotlib.pyplot as plt
for i in range(6):
    io.imshow(Poster[i],interpolation='nearest',aspect='auto')
    plt.show()

In [ ]:
#append list of image array as column to dataframe
Poster_Image_Array=pd.Series(Poster)
#Note that this code inserts alot of info into the dataframe and slows down
#the ability to access values in OgData. 
OgData['Poster_Image_Array']=Poster_Image_Array.values

In [ ]:
OgData.head()

In [ ]:
#Number of Movies by Genre
Unique_Genres=[]
for i in OgData['genres']:
    for y in i:
        if y not in Unique_Genres:
            Unique_Genres.append(y)

In [ ]:
Unique_Genres
#print("The Total Number of Genres is %s" %len(Unique_Genres))

In [ ]:
# counting hhow many occurences of each genre
count_genre=[]
for i in Unique_Genres:
    y=sum(x.count(i) for x in OgData['genres'])
    count_genre.append(y)
count_genre
    

        

In [ ]:
OgData['Poster_Image_Array'][0]

In [ ]:
#OgData['Poster_Image_Array'][column index]
#[2 to access RGB Info for all pixels]
#[individual pixel RGB value][0 is R, 1 is G, 2 is B]
#185 pixels per image
Average_RGB_Values=[]
for j in range(len(OgData['Poster_Image_Array'])):
    R_Values=[]
    G_Values=[]
    B_Values=[]
    if j==2407 or j==2429:
        Average_RGB_Values.append([0,0,0])
    else:
        for y in range(3):
            for i in range(185):
                if y==0: 
                    R_Values.append(OgData['Poster_Image_Array'][j][2][i][y])
                elif y==1:
                    G_Values.append(OgData['Poster_Image_Array'][j][2][i][y])
                elif y==2:
                    B_Values.append(OgData['Poster_Image_Array'][j][2][i][y])
                
        Average_RGB_Values.append([np.mean(R_Values),np.mean(G_Values),np.mean(B_Values)])
    
            
        

In [ ]:
# Rounding up values
import math as m
for i in range (len(Average_RGB_Values)):
    for y in range(len(Average_RGB_Values[i])):
        Average_RGB_Values[i][y]=m.ceil(Average_RGB_Values[i][y])
        

In [ ]:
 Average_RGB_Values

In [ ]:
# Adding Average_RGB_Values to DataFrame 
m=pd.Series(Average_RGB_Values)
OgData['Avg_RGB_Value']=m.values
OgData.head()
#interesting that the first 3 here ave RGB are [0,0,0] since tehre is a 
#an image produced mostly black....

In [ ]:
OgData.head()

In [ ]:
#Image Analysis
import matplotlib.pyplot as plt
from sklearn import svm,metrics

In [ ]:
from skimage.transform import resize
#images need to have the same dimensions before inputting into
#the model
for i in range(len(Poster)):
    
    Poster[i]=resize(Poster[i],(185,185))

In [ ]:
Poster=np.array(Poster)
Poster[0].shape

In [ ]:
Poster.shape

In [ ]:
#image example from sklearn in order to better understand how the 
#analysis is going to work.
from sklearn import datasets
digits=datasets.load_digits()
digits.images.shape
data2=digits.images.reshape((len(digits.images),-1))
data2.shape

In [ ]:
digits.images.shape

In [ ]:
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[:4]):
    plt.subplot(2, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %i' % label)

In [ ]:
#actual model
#since movies have multiple genres,we have to use the 
#MultiLabelBinarizer Hot encode all the genres for for each movie
from sklearn.preprocessing import MultiLabelBinarizer
mlb=MultiLabelBinarizer()
y=mlb.fit_transform([i for i in OgData['genres']])
# these are all the classes for all genres
mlb.classes_

In [ ]:
y_test=[]
for i in range(len(y)):
    if i==2407 or i==2429:
        next
    else:
        y_test.append(y[i])

In [ ]:
y=np.array(y_test)

In [ ]:
n_samples=len(Poster)
data=Poster.reshape((len(Poster),-1))
data.shape
#len(data[:n_samples//2])

In [ ]:
#Support Vector Classification
# notice 50% training set and 50% test set
#WARNING: IT TAKES MORE THAN 1 HOUR TO RUN
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
classifier=OneVsRestClassifier(estimator=svm.LinearSVC(random_state=0))
classifier.fit(data[:n_samples//2],y[:n_samples//2])
#using SVM because it works really well with high dimensions


In [ ]:
expected=y[n_samples//2:]
predicted=classifier.predict(data[n_samples//2:])
%time
# function %time didnt work correctly this code took longer than 30 min to run

In [ ]:
expected=y[n_samples//2:]

In [ ]:
print("Classification report for classifier %s:\n%s\n"%(classifier,metrics.classification_report(expected,predicted)))


In [ ]:
#For the above model we had a terrible prediction rate.
#Factors that could've caused this are that there are too many genres.
#maybe alot of defining features exist in the RGB values rather than in gray-scale


In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(expected,predicted))
#this accuracy represents for each poster it identified all 20 labels correctly 2% of the time.

In [ ]:
BData=OgData
BData

In [ ]:
#Adding in Binary Features
#attempting categorical variables to predict whether a movie was a success
BData['profit']=np.where(OgData['revenue']/OgData['budget']>1,1,0)
BData['Movie_Collection_bool']=np.where(OgData['belongs_to_collection'] == np.nan,0,1)
BData['Number_Genres']=OgData['genres'].apply(lambda x: len(x)) 
BData['Number_Production_Companies']=OgData['production_companies'].apply(lambda x: len(x)) 
BData['Number_Production_Countries']=OgData['production_countries'].apply(lambda x: len(x)) 
for i in Unique_Genres:
    BData['is_'+str(i)]=OgData['genres'].apply(lambda x:1 if i in x else 0)
        

In [ ]:
BData

In [ ]:
#Logistic Regression
BData2=BData.drop(['return','belongs_to_collection','genres','original_language','popularity','poster_path','production_companies','production_countries','release_date','revenue','runtime','spoken_languages','tagline','title','overview'],axis=1)
BData2=BData2.dropna()
BData2

In [ ]:
Predicted=logistic.predict(X_Test)
print("Confusion matrix:\n%s" % metrics.confusion_matrix(Y_Test, Predicted))

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_Test,Predicted))

In [ ]:
# still just an okay accuracy not much better than guessing randomly.